In [1]:
import torch
import numpy as np
import random
import os

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

# Call the function with your chosen seed
seed_everything(seed=1234)


In [2]:
import sys

sys.path.append("/home/sayem/Desktop/deepHSI")  # Adjust to your project root path

from pathlib import Path
import numpy as np

# Custom module imports
from src.dataset.components.hyperspectral_dataset import HyperspectralDataset
from src.dataset.components.utils import *
from src.dataset.medical_datasets.bloodHSI import BloodDetectionHSIDataModule
from src.dataset.remote_sensing_datasets.paviaC import PaviaCDataModule
from src.dataset.remote_sensing_datasets.ksc import KSCDataModule
from src.models.hsi_classification_module import HSIClassificationLitModule
from src.models.components.simple_dense_net import HSIFCModel

# PyTorch and metrics imports
import torch
from torchmetrics import Precision, Recall, F1Score

# Importing from `lightning` instead of `pytorch_lightning`
import lightning as L
# from lightning import Trainer

torch.set_float32_matmul_precision('medium')

In [3]:
from scipy.io import loadmat

# Path to the .mat file
mat_file_path = '/home/sayem/Desktop/deepHSI/data/PaviaC/PaviaC/Pavia_gt.mat'

# Load the .mat file
data = loadmat(mat_file_path)

data

{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Fri May 20 18:26:22 2011',
 '__version__': '1.0',
 '__globals__': [],
 'pavia_gt': array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)}

In [4]:
# Define the parameters for the data module
data_dir = '/home/sayem/Desktop/deepHSI/data'  # Specify the directory where you want the data to be downloaded

# Include 'batch_size', 'num_workers', and 'num_classes' within the hyperparams dictionary
hyperparams = {
    "batch_size": 64,
    "num_workers": 24,
    "patch_size": 10, 
    "center_pixel": True, 
    "supervision": "full",
    "num_classes": 10  # Define the number of classes in your dataset
}

# Assuming YourModel is defined elsewhere and num_classes is known
input_channels = 102

# Define custom metrics for the classification task using the updated hyperparams
custom_metrics = {
    "precision": Precision(num_classes=hyperparams["num_classes"], average='macro', task='multiclass'),
    "recall": Recall(num_classes=hyperparams["num_classes"], average='macro', task='multiclass'),
    "f1": F1Score(num_classes=hyperparams["num_classes"], average='macro', task='multiclass')
}

model = HSIFCModel(
    input_channels=input_channels,
    patch_size=hyperparams["patch_size"],  # Use patch_size from hyperparams
    n_classes=hyperparams["num_classes"],  # Use num_classes from hyperparams
    dropout=True
)

from lightning.pytorch.callbacks import LambdaCallback

# Initialize the HSIClassificationLitModule with the model and other hyperparameters
hsi_module = HSIClassificationLitModule(
    net=model,  # Your model
    optimizer='Adam',  # Using the Adam optimizer
    optimizer_params={"lr": 1e-5},  # Initial learning rate set to 1e-5
    num_classes=hyperparams["num_classes"],  # Number of classes from your hyperparameters
    custom_metrics=custom_metrics,  # Custom metrics if any
    scheduler=None, # 'ReduceLROnPlateau',  # Using the ReduceLROnPlateau learning rate scheduler
    scheduler_params={}
)

# # Initialize the PyTorch Lightning Trainer
# trainer = Trainer(max_epochs=10, precision='16-mixed', accelerator='gpu', devices=1)
max_epochs = 200

# Initialize the PaviaCDataModule with the updated arguments
pavia_c_datamodule = PaviaCDataModule(
    data_dir=data_dir,
    hyperparams=hyperparams  # Pass hyperparams which now includes num_classes
)

# Define the EarlyStopping callback
early_stop_callback = L.pytorch.callbacks.EarlyStopping(
    monitor='val/f1',  # Specify the metric to monitor
    patience=10,  # Number of epochs with no improvement after which training will be stopped
    verbose=True,  # Whether to print logs to stdout
    mode='max',  # In 'min' mode, training will stop when the quantity monitored has stopped decreasing
)

In [5]:
from pathlib import Path

# Specify the directory to save checkpoints
ckpt_dir = Path('/home/sayem/Desktop/deepHSI/notebooks/ckpt')

# Function to clear directory
def clear_directory(path: Path):
    if path.is_dir():
        for item in path.iterdir():
            if item.is_dir():
                clear_directory(item)
                item.rmdir()
            else:
                item.unlink()
    else:
        path.mkdir(parents=True, exist_ok=True)

# Clear and/or create the log and checkpoint directories
clear_directory(ckpt_dir)

In [6]:
from lightning.pytorch.callbacks import ModelCheckpoint

# Define the ModelCheckpoint callback
model_checkpoint = ModelCheckpoint(
    monitor='val/f1',  # Metric to monitor
    dirpath=str(ckpt_dir),  # Convert Path object to string, Directory to save checkpoints
    filename='best-checkpoint-{epoch:02d}-{val/f1:.2f}',  # Checkpoint file name
    save_top_k=1,  # Save only the best checkpoint
    mode='max',  # 'max' because we want to maximize 'val/f1'
    verbose=True,  # Print a message when a new best is found
    auto_insert_metric_name=False  # Prevents metric names being inserted into filename automatically
)

In [7]:
# Initialize the PyTorch Lightning Trainer with fast_dev_run enabled
trainer = L.Trainer(
    fast_dev_run=False,  # Enable fast_dev_run
    precision='16-mixed',  # Use 16-bit precision
    accelerator='auto',  # Specify the accelerator as GPU
    max_epochs = max_epochs,
    log_every_n_steps=3,
    callbacks=[early_stop_callback, model_checkpoint],
)

# # Prepare and set up the data module
pavia_c_datamodule.prepare_data()
pavia_c_datamodule.setup(stage='test')

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/sayem/anaconda3/envs/deepHSI/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default


Dataset 'PaviaC' already exists. Skipping download.


In [ ]:
test_datamodule = 

In [8]:
STOP

NameError: name 'STOP' is not defined

In [ ]:
# # # Prepare and set up the data module
# # pavia_c_datamodule.prepare_data()
# # pavia_c_datamodule.setup(stage='test')  # Use 'test' stage to prepare the test data

# # # Assuming you have a LightningModule (model) defined and initialized
# # # model = YourModel()

# # # Evaluate the model on the test set before training (untrained performance)
# trainer.test(model, datamodule=pavia_c_datamodule)

# # # After testing the untrained model, you can proceed to train the model
# # pavia_c_datamodule.setup(stage='fit')  # If needed, set up for training
# # trainer.fit(model, datamodule=pavia_c_datamodule)

In [ ]:
from lightning.pytorch.tuner.tuning import Tuner

# Create a Tuner object
tuner = Tuner(trainer)

# Use the lr_find method from the Tuner
lr_finder = tuner.lr_find(
    model=hsi_module,
    datamodule=pavia_c_datamodule,
    min_lr=1e-8,  # minimum learning rate to investigate
    max_lr=1e-1,  # maximum learning rate to investigate
    num_training=200,  # number of learning rates to test
    mode='exponential',  # increases the learning rate exponentially
)

# Plot the results
fig = lr_finder.plot(suggest=True)
# fig.show()

# Get the suggested learning rate
new_lr = lr_finder.suggestion()

# Update the learning rate in your model's or optimizer's configuration
hsi_module.hparams.lr = new_lr

In [ ]:
# Use the scale_batch_size method from the Tuner
new_batch_size = tuner.scale_batch_size(
    model=hsi_module,
    datamodule=pavia_c_datamodule,
    mode='power',  # 'power' or 'binsearch' for search strategy
    steps_per_trial=3,  # Number of steps to run with a given batch size
    init_val=hyperparams['batch_size'],  # Start from the current batch size in hyperparams
    max_trials=25,  # Max number of tries before stopping
    batch_arg_name='batch_size'  # The attribute name in model or datamodule that stores the batch size
)

# Update the batch size in hyperparameters
hsi_module.hparams['batch_size'] = new_batch_size
pavia_c_datamodule.batch_size = new_batch_size  # Make sure to update the data module's batch size as well

In [ ]:
# Fit the model using the train dataset from the data module
# trainer.fit(hsi_module, pavia_c_datamodule.train_dataloader())  
trainer.fit(hsi_module, datamodule=pavia_c_datamodule)
# Use train_dataloader() instead of train_dataset